# Algoritmos de Optimización.
Por: Samuel De Dios

# Programación lineal.

Importar libreria.

In [32]:
!pip install pulp
from pulp import *


[notice] A new release of pip is available: 23.3.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Ejemplo min.

Se usa de ejemplo este ejercicio.

\begin{align*}
\text{Minimizar }&19x_{1}+57x_{2} \\
\text{Restric. }& x_{1} + 2x_{2} \geq 10\\
& x_{1} + 4x_{2} \geq 24\\
& 3x_{1} + 2x_{2} \geq 20\\
& 5x_{1} + x_{2} \geq 25\\
\end{align*}

Se define un objeto problema, luego se pone nombre y lo que se quiere hacer (minimizar o maximizar), luego se definen las variables del problema.

In [33]:
problema = LpProblem("proLineal", LpMinimize)
x1 = LpVariable("x1", lowBound = 0)
x2 = LpVariable("x2", lowBound = 0)

Se agrega primero la función objetivo y despues las restricciones.

In [34]:
problema += 19*x1 + 57*x2
problema += x1 + 2*x2 >= 10
problema += x1 + 4*x2 >= 24
problema += 3*x1 + 2*x2 >= 20
problema += 5*x1 + x2 >= 25

Se solucciona el problema.

In [35]:
problema.solve()

print(f"x1 = {x1.varValue} y x2 = {x2.varValue}, con valor optimo {value(problema.objective)}")

x1 = 4.0 y x2 = 5.0, con valor optimo 361.0


### Ejemplo max.

\begin{align*}
\text{Minimizar }&10x_{1} + 24x_{2} + 20x_{3} + 35x_{4} \\
\text{Restric. }& x_{1} + x_{2} + 3x_{3} + 5x_{4} \leq 19\\
& 2x_{1} + 4x_{2} + 2x_{3} + x_{4} \leq 57\\
\end{align*}

Definicion del problema.

In [36]:
problema = LpProblem("prolineal", LpMaximize)
x1 = LpVariable("x1", lowBound = 0)
x2 = LpVariable("x2", lowBound = 0)
x3 = LpVariable("x3", lowBound = 0)
x4 = LpVariable("x4", lowBound = 0)

problema += 10*x1 + 24*x2 + 20*x3 + 35*x4
problema += x1 + x2 + 3*x3 + 5*x4 <= 19
problema += 2*x1 + 4*x2 + 2*x3 + x4 <= 57

In [37]:
problema.solve()

print(f"x1 = {x1.varValue}, x2 = {x2.varValue}, x3 = {x3.varValue} y x4 = {x4.varValue}, con valor optimo {value(problema.objective)}")

x1 = 0.0, x2 = 14.0, x3 = 0.0 y x4 = 1.0, con valor optimo 371.0


# Optimización sin restricciones.

## Librerías.

In [38]:
import math
import sympy as sp
import numpy as np
import scipy as sci

## Optimización de una variable.

### Funciones basicas.

In [39]:
def evaluar_func(funcion, values):
  #Evalua la funcion con los valores
  return eval(funcion, vars(math), values)

def derivate(funcion, var = ['x'], num = [1]):
  #recibe una funcion, una lista de variables con las que derivara y el numero de veces de cada una
  for i, v in enumerate(var):
    funcion = str(sp.diff(funcion, v, num[i]))
  return funcion

def find_in_list(target, py_list):
  #Encuentra target en py_list
  for i in range(len(py_list)):
    if py_list[i] == target:
      return i

### Método de sección dorada.
Es necesario darle a la función la funcion objetivo (`funcion`), un intervalo donde buscar (`cota_inf` y `cota_sup`) y por ultimo una tolerancia `h` que esta predefinida en $0.01$.

In [40]:
def sec_dorada(funcion, cota_inf, cota_sup, h = 0.01):
  lam = "a + (1 - (((5**(1 / 2)) - 1) / 2)) * (b - a)"
  miu = "a + ((((5**(1 / 2)) - 1) / 2) * (b - a))"

  lam_ini = evaluar_func(lam, {"a":cota_inf, "b":cota_sup})
  miu_ini = evaluar_func(miu, {"a":cota_inf, "b":cota_sup})

  funcion_li = evaluar_func(funcion, {"x":lam_ini})
  funcion_mi = evaluar_func(funcion, {"x":miu_ini})

  row = [1, cota_inf, cota_sup, lam_ini, miu_ini, funcion_li, funcion_mi, (cota_sup - cota_inf)]

  while row[7] >= h:
    #print(row)
    if row[5] >= row[6]:
      row[1] = row[3]
    else:
      row[2] = row[4]

    row[3] = evaluar_func(lam, {"a":row[1], "b":row[2]})
    row[4] = evaluar_func(miu, {"a":row[1], "b":row[2]})
    row[5] = evaluar_func(funcion, {"x":row[3]})
    row[6] = evaluar_func(funcion, {"x":row[4]})
    row[7] = row[2] - row[1]
    row[0] += 1

    opti = (row[1] + row[2]) / 2

  return (opti, evaluar_func(funcion, {'x':opti}))

In [41]:
print(f'el punto óptimo es: {sec_dorada("e**x+sin(x)", -4, 4, h = 0.0001)}')

el punto óptimo es: (-1.746147178353358, -0.8102206372964167)


In [42]:
print(f'el punto óptimo es: {sec_dorada("sin(x)*e**x", -2, 1, h = 0.001)}')

el punto óptimo es: (-0.7854243825048068, -0.32239694172320926)


### Método de búsqueda de Fibonacci.
Es necesario darle a la función la funcion objetivo (`funcion`), un intervalo donde buscar (`cota_inf` y `cota_sup`) y por ultimo una tolerancia `h` que esta predefinida en $0.01$.

In [43]:
def fibonacci(ext):
  lista = [1, 1]

  while lista[-1] <= ext:
    lista.append(lista[-1] + lista[-2])

  return lista

In [44]:
def bus_fibonacci(funcion, cota_inf, cota_sup, h = 0.01):
    inc = h / 10
    fi = fibonacci((cota_sup - cota_inf) / h)
    n = len(fi)

    lam = "a + (1 - (f/g)) * (b - a)"
    miu = "a + ((f/g) * (b - a))"

    lam_ini = evaluar_func(lam, {"a":cota_inf, "b":cota_sup, "f":fi[-2], "g":fi[-1]})
    miu_ini = evaluar_func(miu, {"a":cota_inf, "b":cota_sup, "f":fi[-2], "g":fi[-1]})

    row = [1, cota_inf, cota_sup, lam_ini, miu_ini, evaluar_func(funcion, {"x":lam_ini}), evaluar_func(funcion, {"x":miu_ini}), (cota_sup - cota_inf)]

    for i in range(1, n + 1):
      #print(row)

      if row[5] >= row[6]:
        row[1] = row[3]
      else:
        row[2] = row[4]

      row[3] = evaluar_func(lam, {"a":row[1], "b":row[2], "f":fi[n-i-1], "g":fi[n-i]})
      row[4] = evaluar_func(miu, {"a":row[1], "b":row[2], "f":fi[n-i-1], "g":fi[n-i]})

      if i != n-2:
        row[5] = evaluar_func(funcion, {"x":row[3]})
        row[6] = evaluar_func(funcion, {"x":row[4]})
      else:
        row_2 = [i, row[1], row[2], evaluar_func(lam, {"a":row[1], "b":row[2], "f":fi[n-i-1], "g":fi[n-i]}), evaluar_func(lam, {"a":row[1], "b":row[2], "f":fi[n-i-1], "g":fi[n-i]}) + inc]

        if evaluar_func(funcion, {"x":row_2[3]}) > evaluar_func(funcion, {"x":row_2[4]}):
          row_2[1] = row_2[3]
        else:
          row_2[2] = row_2[4]


      row[7] = row[2] - row[1]
      row[0] += 1
    opti = (row[1] + row[2]) / 2

    return (opti, evaluar_func(funcion, {'x':opti}))

In [45]:
print('el punto óptimo es:', bus_fibonacci("e**x+sin(x)", -4, 4, h = 0.0001))

el punto óptimo es: (-1.7461332462908614, -0.8102206373074288)


In [46]:
print('el punto óptimo es:', bus_fibonacci("6*e**(-0.1*x)+2*x", -20, 20, h = 0.0001))

el punto óptimo es: (-12.039685592068945, -4.0794560863385065)


### Método de búsqueda de bisección.
Es necesario darle a la función la funcion objetivo (`funcion`), un intervalo donde buscar (`cota_inf` y `cota_sup`) y por ultimo una tolerancia `h` que esta predefinida en $0.01$. Este metodo usa la primera derivada de la funcion para encontrar en mínimo.

In [47]:
def bus_biseccion(funcion, cota_inf, cota_sup, h = 0.01):
  deri = derivate(funcion)

  row = [1, cota_inf, cota_sup, (cota_sup + cota_inf)/2, evaluar_func(deri, {'x':(cota_sup + cota_inf)/2}), cota_sup - cota_inf]

  while row[-1] >= h and row[4] != 0:
    #print(row)
    row[0] += 1

    if row[4] > 0:
      row[2] = row[3]
    else:
      row[1] = row[3]

    row[3] = (row[1] + row[2]) / 2
    row[4] = evaluar_func(deri, {'x':row[3]})

    row[-1] = row[2] - row[1]

  return (row[3], evaluar_func(funcion, {'x':row[3]}))

In [48]:
print('el punto óptimo es:', bus_biseccion("e**x+sin(x)", -4, 4, h = 0.0001))

el punto óptimo es: (-1.746124267578125, -0.8102206371953053)


In [49]:
print('el punto óptimo es:', bus_biseccion("6*e**(-0.1*x)+2*x", -20, 20, h = 0.0001))

el punto óptimo es: (-12.039756774902344, -4.079456086436171)


### Método de Newton.
Es necesario darle a la función la funcion objetivo (`funcion`), un valor `ini` que sera el valor inicial y por ultimo una tolerancia `h` que esta predefinida en $0.0001$. Este metodo usa las dos primeras derivada de la funcion para encontrar en mínimo.

In [50]:
def met_newton_u(funcion, ini, h = 0.0001):
  deri = derivate(funcion)
  deri_s = derivate(deri)

  row = [1, ini, evaluar_func(deri, {'x':ini}), evaluar_func(deri_s, {'x':ini})]
  row.append(row[1] - (row[2]/row[3]))

  while abs(row[2]) >= h and abs(row[-1] - row[1]) >= h:
    #print(row)
    row[0] += 1

    row[1] = row[-1]
    row[2] = evaluar_func(deri, {'x':row[1]})
    row[3] = evaluar_func(deri_s, {'x':row[1]})
    row[4] = row[1] - (row[2]/row[3])

  return row[-1], evaluar_func(funcion, {'x':row[-1]})

In [51]:
print('el punto óptimo es: ',met_newton_u("e**x+sin(x)", 2, 0.00000001))

el punto óptimo es:  (-1.7461395304080125, -0.8102206373303156)


### Función a trozos.

In [52]:
def opti_trozos(funciones, intervalos, metodo, h=0.001):
  soluciones = []
  f = []
  for i in range(len(funciones)):
    soluciones.append(metodo(funciones[i], intervalos[i][0], intervalos[i][1], h))
    f.append(soluciones[-1][1])

  return soluciones[find_in_list(min(f),f)]

In [53]:
print('el punto óptimo es: ', opti_trozos(["(x+3)/(x-2)", "-(x+1)/x"], [(-20,1),(1,20)], sec_dorada))

el punto óptimo es:  (0.9995710083354113, -3.997855961451723)


In [54]:
print('el punto óptimo es: ', opti_trozos(["(x+3)/(x-2)", "-(x+1)/x"], [(-20,1),(1,20)], bus_fibonacci))

el punto óptimo es:  (0.9997735507249135, -3.998868009962891)


## Optimización de varias variables.
En vez de usar variables como $x$, $y$, $z$, etc., se debe de poner las funciones en terminos de $x_{1}, x_{2}, x_{3} ...$

E.j.  $f(\vec{x})=x_{1}x_{2}+\frac{\sin(x_3)}{x_{4}}$

### Funciones basicas para varias variables.

In [55]:
def evaluar_vec(funcion, vec):
  return evaluar_func(funcion, {'x' + str(i+1):vec[i] for i in range(len(vec))})

In [56]:
#ambas funciones retornan una tupla, sin importar si se ingresa una lista o tupla
def suma(vec1, vec2):
  return tuple(vec1[i] + vec2[i] for i in range(len(vec1)))

def multi(const, vec):
  return tuple(const * vec[i] for i in range(len(vec)))

def suma_var(vec1, vec2):
  return tuple(f'{vec1[i]} + {vec2[i]}' for i in range(len(vec1)))

def multi_var(const, vec):
  return tuple(f'({const}*{str(vec[i])})' for i in range(len(vec)))

def evaluar_vec_var(funcion, vec, variables):
  for i in range(len(vec)):
    funcion = funcion.replace(variables[i], f'({vec[i]})')
  return funcion

### Método de Nelder y Mead.

In [57]:
def paso5(funcion, lista_p, f_p, puntos, long):
  for i in range(len(lista_p)):
    lista_p[i] = suma(puntos[1], multi(long, suma(lista_p[i], multi(-1, puntos[1]))))
    f_p[i] = evaluar_vec(funcion, lista_p[i])

  return lista_p, f_p

In [58]:
def met_nelder_mead(funcion, x1, tol = 0.0001, reflexion = 1, contraccion = 0.5, expansion = 2, long_paso = 0.5):
  #paso 0
  #agregar nuevos puntos y tomar sus valores de f
  lista = [x1]
  fs = [evaluar_vec(funcion, x1)]

  e = [0] * len(x1)
  for i in range(len(x1)):
    e[i] = 1
    poi = suma(x1, multi(long_paso, e))
    lista.append(poi)
    fs.append(evaluar_vec(funcion, poi))
    e[i] = 0

  while True:
    #paso 1
    #ordenar
    index_min = find_in_list(min(fs), fs)
    index_max = find_in_list(max(fs), fs)

    lista_sin = lista[:index_max] + lista[index_max+1:]
    fs_sin = fs[:index_max] + fs[index_max+1:]
    index_maxm = find_in_list(max(fs_sin), fs_sin)
    if lista[index_maxm] != lista_sin[index_maxm]:
      index_maxm += 1

    points = [lista[index_min], lista[index_maxm], lista[index_max]]
    del lista_sin, fs_sin
    fp = [evaluar_vec(funcion, p) for p in points]

    #promedio
    promedio = 0
    for f in fs:
      promedio += f
    promedio /= len(fs)

    #varianza
    varianza = 0
    for f in fs:
      varianza += (f - promedio)**2
    varianza /= len(fs)

    if varianza**(1/2) <= tol:
      return points[0]

    #paso 2
    #centroide
    x0 = [0]*len(x1)
    for i in lista:
      if i != points[-1]:
        x0 = suma(x0, i)
    x0 = multi(1/(len(lista)-1), x0)

    #punto de reflexion
    xr = suma(multi(1+reflexion, x0), multi((-1)*reflexion, points[-1]))
    fr = evaluar_vec(funcion, xr)

    if fp[0] <= fr < fp[1]:
      lista[index_max] = xr
      fs[index_max] = fr

    elif fr < fp[0]:
      #Paso 3
      xe = suma(multi(1+(reflexion*expansion), x0), multi((-1)*reflexion*expansion, points[-1]))
      fe = evaluar_vec(funcion, xe)

      if fe < fr:
        lista[index_max] = xe
        fs[index_max] = fe

      else:
        lista[index_max] = xr
        fs[index_max] = fr

    elif fr >= fp[1]:
      #Paso 4
      if fp[1] <= fr < fp[-1]:
        xce = suma(multi(1+(reflexion*contraccion), x0), multi((-1)*reflexion*contraccion, points[-1]))
        fce = evaluar_vec(funcion, xce)

        if fce <= fr:
          lista[index_max] = xce
          fs[index_max] = fce

        else:
          lista, fs = paso5(funcion, lista, fs, points, long_paso)

      elif fr >= fp[-1]:
        xci = suma(multi(1-contraccion, x0), multi(contraccion, points[-1]))
        fci = evaluar_vec(funcion, xci)

        if fci < fp[-1]:
          lista[index_max] = xci
          fs[index_max] = fci

        else:
          lista, fs = paso5(funcion, lista, fs, points, long_paso)

    else:
      lista, fs = paso5(funcion, lista, fs, points, long_paso)

In [59]:
print('el punto óptimo es: ',met_nelder_mead('x1**2 - x1*x2 + x2**2 - 3*x2', (0,0), tol = 0.0001))

el punto óptimo es:  (0.9968459606170654, 2.011179208755493)


In [60]:
print('el punto óptimo es: ',met_nelder_mead('((x1-2)**2 + x2**2)**(1/2)', (0,0), tol = 0.0001))

el punto óptimo es:  (1.9992618359574408, -0.0007397923391181394)


### Método  del descenso más empinado.

In [61]:
from scipy.optimize import minimize_scalar
from sympy import lambdify
def descenso(funcion, x1, tol = 0.0001):
  vars = ["x"+str(i+1) for i in range(len(x1))]

  gradiente = []
  norma = ''
  e = [0] * len(x1)
  for i in range(len(x1)):
    e[i] = 1
    gradiente.append(derivate(funcion, vars, e))
    norma += f'({gradiente[i]})**2'
    if i != len(x1) - 1:
      norma += ' + '
    e[i] = 0

  while evaluar_vec(norma, x1)**(1/2) >= tol:
    dk = multi(-1, [evaluar_vec(i, x1) for i in gradiente])
    try:
      lam_k = float(minimize_scalar(lambdify('x', evaluar_vec_var(funcion, suma_var(x1, multi_var('x', dk)), vars)), method='brent').x)
    except:
      return x1

    x1 = suma(x1, multi(lam_k, dk))

  return x1

In [62]:
print('el punto óptimo es: ',descenso('(8 - x1*x2)**2 + 7*(x2 - x1**2)**2', (0,3), 0.0001))

el punto óptimo es:  (1.9999989622799033, 3.9999949033711983)


### Método de Newton.
Tiene problemas con la matriz hessiana, ya que no siempre es invertible. Y a su vez puede dar un punto mínimo, máximo o punto de silla.

In [67]:
from numpy.linalg.linalg import LinAlgError
def met_newton(funcion, x1, tol = 0.0001):
  vars = ["x"+str(i+1) for i in range(len(x1))]

  while True:
    gradiente = []
    norma = ''
    e = [0] * len(x1)
    for i in range(len(x1)):
      e[i] = 1
      gradiente.append(derivate(funcion, vars, e))
      norma += f'({gradiente[i]})**2'
      if i != len(x1) - 1:
        norma += ' + '
      e[i] = 0

    norma_gr = evaluar_vec(norma, x1)

    if norma_gr**(1/2) < tol:
      return x1

    e = [0] * len(x1)
    h = []
    for i in range(len(gradiente)):
      h.append([])
      for j in range(len(x1)):
        e[j] = 1
        h[i].append(evaluar_vec(derivate(gradiente[i], vars, e), x1))
        e[j] = 0


    hessiana = np.array(h)
    try:
      hessiana_inv = np.linalg.inv(hessiana)
    except LinAlgError:
      print("La matriz hessiana no tiene inversa.")
      print(hessiana)
      return x1
    gradiente_k = [evaluar_vec(i, x1) for i in gradiente]
    x2 = suma(x1, multi(-1, hessiana_inv.dot(gradiente_k)))
    #print(x2)

    n = 0
    xk1 = suma(x2, multi(-1, x1))
    for i in xk1:
      n+=(i**2)

    if n**(1/2) < tol:
      return x2
    x1 = x2

In [68]:
print('el punto óptimo es: ',met_newton('(8 - x1*x2)**2 + 7*(x2 - x1**2)**2', (1,1)))

el punto óptimo es:  (-0.3758940646715096, -0.2825926855561822)


In [69]:
print('el punto óptimo es: ',met_newton('(x1*x2)**(1/2)', (1,1)))

La matriz hessiana no tiene inversa.
[[-0.25  0.25]
 [ 0.25 -0.25]]
el punto óptimo es:  (1, 1)


### Método de Newton-CG.

In [71]:
def met_newton_cg(funcion, x1, tol = 0.0001):
  vars = ["x"+str(i+1) for i in range(len(x1))]

  gradiente = []
  norma = ''
  e = [0] * len(x1)
  for i in range(len(x1)):
    e[i] = 1
    gradiente.append(derivate(funcion, vars, e))
    norma += f'({gradiente[i]})**2'
    if i != len(x1) - 1:
      norma += ' + '
    e[i] = 0


  e = [0] * len(x1)
  h = []
  for i in range(len(gradiente)):
    h.append([])
    for j in range(len(x1)):
      e[j] = 1
      h[i].append(derivate(gradiente[i], vars, e))
      e[j] = 0
  hf = h.copy()

  while evaluar_vec(norma, x1)**(1/2) >= tol:
    g1 = np.array([evaluar_vec(i, x1) for i in gradiente])
    d1 = np.array(multi(-1, g1))


    for i in range(len(x1)):

      for k in range(len(h)):
        for j in range(len(h[k])):
          hf[k][j] = evaluar_vec(str(h[k][j]), x1)

      hessiana = np.array(hf)
      lam_k = -(g1.transpose().dot(d1))/(d1.dot(hessiana).dot(d1.transpose()))
      beta_k = (g1.transpose().dot(hessiana).dot(d1))/(d1.transpose().dot(hessiana).dot(d1))


      x1 = suma(x1, multi(lam_k, d1))
      g1 = np.array([evaluar_vec(i, x1) for i in gradiente])

      if i >= len(x1) - 1:
        d1 = np.array(suma(multi(-1, g1), multi(beta_k, d1)))


  return x1

In [72]:
print('el punto óptimo es:',met_newton_cg('(x1 - 2)**4 + (x1 - 2*x2)**2', (0,3), 0.00001))

el punto óptimo es: (2.0140861525652727, 1.007043635383157)


In [73]:
print('el punto óptimo es:',met_newton_cg('(8 - x1*x2)**2 + 7*(x2 - x1**2)**2', (2,2)))

el punto óptimo es: (1.9999993157899167, 3.9999961722318913)


# Optimización con restricciones.

## Funciones de penalización.

Si las restricciones son de igualdad se puede usar funciones de penalización para encontrar un valor que cumpla las restricciones.

Ejemplo:
\begin{align*}
\text{Minimizar }&x_{1}^{2} + x_{2}^{2} \\
\text{Restric. }& x_{1} + x_{2} = 1
\end{align*}
Se puede rescribir como una funcion sin restricciones de la forma:
\begin{align*}
\text{Minimizar }&x_{1}^{2} + x_{2}^{2} + M\left(x_{1} + x_{2} - 1\right)^{2}
\end{align*}
Con $M$ un número muy grande.

In [76]:
def min_penalizacion_igualdad(funcion, restri_igualdad, x0):
  restri = ""

  for i in range(len(restri_igualdad)):
    restri += f"9999999999*(({restri_igualdad[i]})**2"
    if i < len(restri_igualdad)-1:
      restri+="+"
  return met_nelder_mead(funcion + f"+ {restri})", x0)

In [77]:
min_penalizacion_igualdad("x1**2 + x2**2", ["x1 + x2 - 1"], (0,0))

(0.5, 0.5)

In [78]:
min_penalizacion_igualdad("x1**2 + 2*x2**2 + 3*x3**2 - 4*x2*x3 - 4*x1 - 2*x2", ["x1 + x2 + x3 - 10"], (0,0,0))

(5.30700688616745, 2.813678019680739, 1.879315177993766)

Si las restricciones son de desigualdad se usa el siguiente algoritmo, se deben de tener las desigualdades de la forma $\leq$

In [ ]:
def min_penalizacion_des(funcion, restri_desigualdad, x0, tol=0.001):
  u=0.1
  beta=10
  funcion_aux=funcion

  vars=[f"x{i+1}" for i in range(len(x0))]

  for i in restri_desigualdad:
    funcion_aux = funcion_aux + f"+{u}*(max(0,{i}))**2"

  x0 = met_nelder_mead(funcion_aux,x0,tol=tol)

  func = lambdify(vars, funcion_aux)
  determinante = lambdify(vars,funcion_aux[len(funcion)+1:])

  while determinante(*list(x0))>tol:
    funcion_aux=funcion
    for i in restri_desigualdad:
      funcion_aux = funcion_aux + f"+{u}*(max(0,{i}))**2"

    x0 = met_nelder_mead(funcion_aux,x0,tol=tol)

    determinante = lambdify(vars,funcion_aux[len(funcion)+1:])
    u*=beta

  return x0

In [ ]:
min_penalizacion_des("-(x1 + 3*x2)", ["2*x1 + 3*x2 - 6", "(-1)*x1 + 4*x2 - 4", "(-1)*x1", "(-1)*x2"], (1,1))

Por ultimo se unen las dos funciones, tanto de igualdades como desigualdades.

In [85]:
def min_penalizacion(funcion, restri_igualdad, restri_desigualdad, x0, tol=0.00001):
  #las restricciones deben de ser listas, las de desigualdad =<0
  #y las igualdades de forma =0
  u=0.1
  beta=10
  funcion_aux=funcion
  vars=[f"x{i+1}" for i in range(len(x0))]

  for i in restri_igualdad:
    funcion_aux = funcion_aux + f"+{u}*({i})**2"

  for i in restri_desigualdad:
    funcion_aux = funcion_aux + f"+{u}*(max(0,{i}))**2"

  x0 = met_nelder_mead(funcion_aux,x0,tol=tol)

  determinante = lambdify(vars,funcion_aux[len(funcion)+1:])

  while determinante(*list(x0))>tol:
    funcion_aux=funcion

    for i in restri_igualdad:
      funcion_aux = funcion_aux + f"+{u}*({i})**2"

    for i in restri_desigualdad:
      funcion_aux = funcion_aux + f"+{u}*(max(0,{i}))**2"

    x0 = met_nelder_mead(funcion_aux,x0,tol=tol)

    determinante = lambdify(vars,funcion_aux[len(funcion)+1:])
    u*=beta

  return x0

In [86]:
print(min_penalizacion("(x1-2)**4 + (x1-2*x2)**2", [], ["x1**2 - x2"], (1,2)))

(0.9452894570963508, 0.893576229454311)


In [87]:
min_penalizacion("x1**2+2*x2**2+3*x3**2-4*x2*x3-4*x1-2*x2", ["x1+x2+x3-10"], [], (8,1,1))

(2.997965323016637, 4.002370778585202, 2.999664385236164)

## Función barrera.

In [88]:
def min_barrera(funcion, restri_desigualdad, x0, tol=0.001):
  #las restricciones deben de ser listas, las de desigualdad =<
  u1=10
  betas=0.1
  funcion_aux=funcion
  vars=[f"x{i+1}" for i in range(len(x0))]

  for i in restri_desigualdad:
    funcion_aux = funcion_aux + f"-({u1}/({i}))"

  x0 = min_penalizacion(funcion_aux,[], restri_desigualdad, x0, tol=tol)

  determinante = lambdify(vars,funcion_aux[len(funcion):])

  while abs(determinante(*list(x0)))>tol:
    u1*=betas
    funcion_aux=funcion

    for i in restri_desigualdad:
      funcion_aux = funcion_aux + f"-({u1}/({i}))"

    try:
      x0 = min_penalizacion(funcion_aux,[], restri_desigualdad, x0, tol=tol)
    except ZeroDivisionError:
      x0=min_penalizacion(funcion,[], restri_desigualdad, x0, tol=tol)
      break

    determinante = lambdify(vars, funcion_aux[len(funcion):])


  return x0

In [89]:
min_barrera("(x1-2)**4 + (x1-2*x2)**2", ["x1**2 - x2"], (1,2), 0.0001)

(0.9479080905502437, 0.898448182579477)